# pattern for annex with multiple tables and entity types

In [1]:
#table annex with numbers
import pandas
import os
import requests
import re
from bs4 import BeautifulSoup

url = 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32014D0499'
response = requests.get(url)
print(response.status_code)
data = response.text
data

200


'<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML//EN" "xhtml-strict.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><!-- CONVEX # converter_version:9.15.0 # generated_on:20240306-1652 # ELI version:0.10 --><head><meta name="format-detection" content="telephone=no"/><meta http-equiv=\'X-UA-Compatible\' content=\'IE=edge\'/><meta name=\'viewport\' content=\'width=device-width, initial-scale=1\'><meta name=\'WT.z_usr_lan\' content=\'EN\'>\n      <meta http-equiv="content-type" content="text/html; charset=utf-8"/>\n      <script type="text/javascript" src="/ruxitagentjs_ICANVfgqrux_10289240325103055.js" data-dtconfig="app=47d4c64c3b67ec69|agentId=daf5c5adbd7ceee|featureHash=ICANVfgqrux|rdnt=1|uxrgce=1|bp=3|cuc=m097nmfl|mel=100000|mb=null|dpvc=1|iub=null|ssv=4|lastModification=1721139136808|tp=500,50,0|agentUri=/ruxitagentjs_ICANVfgqrux_10289240325103055.js|reportUrl=/rb_39a3e95b-5423-482c-879b-99ef235dffeb|rid=RID_1236501551|rpid=-1791395488|domain=europa.eu"

In [16]:
#get the document information
url= re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data)
publication_date = re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
title = re.findall(r'class="oj-doc-ti"[^>]*>([\s\S]*?)<\/p>', data)
celex= re.findall(r'uri=CELEX%([^"]+)', data)
decision_match = re.findall(r'(COUNCIL.*?)</p', data)
legal_decision = re.findall(r'<p[^>]*>(.*?(amending|implementing).*?)</p>',data)
legal_base = re.findall(r'<p[^>]*>(.*?regard to.*?)</p>',data)
print(url),print(publication_date),print(title),print(celex),print(regulation_match),print(legal_decision),print(legal_base)

['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32014D0499']
['25.7.2014']
['\n            COUNCIL DECISION 2014/499/CFSP', 'of 25 July 2014\n         ', 'amending Decision 2014/145/CFSP concerning restrictive measures in respect of actions undermining or threatening the territorial integrity, sovereignty and independence of Ukraine', 'ANNEX', 'LIST OF PERSONS AND ENTITIES REFERRED TO IN ARTICLE 2']
['3A32014D0499']
['COUNCIL DECISION 2014/499/CFSP', 'COUNCIL OF THE EUROPEAN UNION,']
[('amending Decision 2014/145/CFSP concerning restrictive measures in respect of actions undermining or threatening the territorial integrity, sovereignty and independence of Ukraine', 'amending'), ('natural persons responsible for, actively supporting or implementing, actions or policies which undermine or threaten the territorial integrity, sovereignty and independence of Ukraine, or stability or security in Ukraine, or which obstruct the work of international organisations in Ukraine,

(None, None, None, None, None, None, None)

In [5]:
#search for UN to add the UN dummy indicator
UN_pattern = re.search(r'UN',data)

UN = None
for i in data:
    if UN_pattern == None:
        UN = 0
    else:
        UN = 1

In [20]:
# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Find all rows in the table
rows = soup.find_all('tr', class_='oj-table')

# List to store the parsed data
sanction_list = []

# Iterate through the rows and extract the necessary information
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) < 5:
        continue

    name = cells[1].get_text(strip=True)
    identifying_info = cells[2].get_text(strip=True)  # Extracting the identifying information
    reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
    date_of_listing = cells[4].get_text(strip=True)
    
    sanction_list.append({
        'URL': url,
        'CELEX':celex,
        'law_number' :decision_match[0],
        'Publication Date': publication_date,
        'legal_base' : legal_decision[0],
        'Name': name,
        'Identifying Information': identifying_info,
        'Reason': reasons,
        'Date of Listing': date_of_listing
    })

# Display the results
for person in sanction_list:
    print(person)

{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32014D0499'], 'CELEX': ['3A32014D0499'], 'law_number': 'COUNCIL DECISION 2014/499/CFSP', 'Publication Date': ['25.7.2014'], 'legal_base': ('amending Decision 2014/145/CFSP concerning restrictive measures in respect of actions undermining or threatening the territorial integrity, sovereignty and independence of Ukraine', 'amending'), 'Name': 'Mikhail Efimovich FRADKOVМихаил Ефимович Фрадков', 'Identifying Information': 'Born on 1.9.1950 in Kurumoch, Kuibyshev region', 'Reason': 'Permanent member of the Security Council of the Russian Federation; Director of the Foreign Intelligence Service of the Russian Federation. As a member of the Security Council, which provides advice on and coordinates national security affairs, he was involved in shaping the policy of the Russian Government threatening the territorial integrity, sovereignty and independence of Ukraine.', 'Date of Listing': '25.7.2014'}
{'URL': ['https://e

In [8]:
sanction_list

[{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32014D0499'],
  'CELEX': ['3A32014D0499'],
  'law_number': '\n            COUNCIL DECISION 2014/499/CFSP',
  'Publication Date': ['25.7.2014'],
  'legal_base': 'amending Decision 2014/145/CFSP concerning restrictive measures in respect of actions undermining or threatening the territorial integrity, sovereignty and independence of Ukraine',
  'Name': 'Mikhail Efimovich FRADKOVМихаил Ефимович Фрадков',
  'Identifying Information': 'Born on 1.9.1950 in Kurumoch, Kuibyshev region',
  'Reason': 'Permanent member of the Security Council of the Russian Federation; Director of the Foreign Intelligence Service of the Russian Federation. As a member of the Security Council, which provides advice on and coordinates national security affairs, he was involved in shaping the policy of the Russian Government threatening the territorial integrity, sovereignty and independence of Ukraine.',
  'Date of Listing': '25.7.2014'},
 